<a href="https://colab.research.google.com/github/vikaskapur/unsupervised-passage-reranking/blob/my-test/Zeroshot_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [1]:
! git clone https://github.com/vikaskapur/unsupervised-passage-reranking

Cloning into 'unsupervised-passage-reranking'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 49 (delta 16), reused 35 (delta 7), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [2]:
%cd unsupervised-passage-reranking

/content/unsupervised-passage-reranking


In [3]:
!python utils/download_data.py --resource "data.wikipedia-split.psgs_w100"

Requested resource from %s https://www.dropbox.com/s/bezryc9win2bha1/psgs_w100.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/unsupervised-passage-reranking/downloads/data/wikipedia-split/psgs_w100.tsv

Downloaded to %s /content/unsupervised-passage-reranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
Uncompressing %s /content/unsupervised-passage-reranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
 Saved to %s ./downloads/data/wikipedia-split


In [4]:
!python utils/download_data.py --resource "data.retriever-outputs.bm25.nq-test" 

Requested resource from %s https://www.dropbox.com/s/ml2lnt34ktjgft6/nq-test.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/unsupervised-passage-reranking/downloads/data/retriever-outputs/bm25/nq-test.json

Downloaded to %s /content/unsupervised-passage-reranking/downloads/data/retriever-outputs/bm25/nq-test.tar.gz
Uncompressing %s /content/unsupervised-passage-reranking/downloads/data/retriever-outputs/bm25/nq-test.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/bm25


In [6]:
!pip install datasets evaluate transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 4.0 MB/s 
     |████████████████████████████████| 69 kB 7.8 MB/s 
     |████████████████████████████████| 4.9 MB 60.1 MB/s 
     |████████████████████████████████| 115 kB 54.2 MB/s 
     |████████████████████████████████| 120 kB 66.9 MB/s 
     |████████████████████████████████| 212 kB 58.8 MB/s 
     |████████████████████████████████| 127 kB 54.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.8 MB/s 
     |████████████████████████████████| 1.3 MB 49.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [8]:
hf_model_name = "bigscience/T0_3B"

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(hf_model_name)
model = T5ForConditionalGeneration.from_pretrained(hf_model_name)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

In [ ]:
for param in model.parameters():
  param.requires_grad = False

# disable dropout
model.eval()

In [ ]:
def process_samples_from_single_path(filename):
    print_rank_0(' > Processing {} ...'.format(filename))
    total = 0
    rows = []
    id2text = {}

    with open(filename) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        next(reader, None)  # skip the headers
        for row in reader:
            # file format: doc_id, doc_text, title
            doc_id = int(row[0])
            text = row[1]
            title = row[2]
            rows.append({'doc_id': doc_id,
                          'text': text,
                          'title': title})

            assert doc_id not in id2text
            id2text[doc_id] = (text, title)
            total += 1
            if total % 100000 == 0:
                print_rank_0('  > processed {} rows so far ...'.format(total))

    print_rank_0(' >> processed {} samples.'.format(len(rows)))

    return rows, id2text